# Modelo de Machile Learning com multiplas saidas

In [2]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

In [8]:
base = pd.read_csv("games.csv")

In [11]:
base.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN


In [10]:
base = base.drop("Other_Sales", axis=1)
base = base.drop("Global_Sales", axis=1)
base = base.drop("Developer", axis=1)


In [12]:
base.shape

(16719, 13)

# Pré-Processamento do Dataset para aplicar o aprendizado de Maquina

### Correção dos valores faltantes

In [13]:
base = base.dropna(axis=0)

In [15]:
base.shape

(6825, 13)

In [21]:
base = base.loc[base["NA_Sales"] > 1]
base = base.loc[base["EU_Sales"] > 1]
base.shape

(258, 13)

In [22]:
base["Name"].value_counts()

Grand Theft Auto V                            4
The Elder Scrolls V: Skyrim                   3
Assassin's Creed IV: Black Flag               3
Call of Duty: Ghosts                          3
Battlefield 4                                 3
                                             ..
Kingdom Hearts                                1
The Legend of Zelda: A Link Between Worlds    1
Sonic Mega Collection Plus                    1
Destiny                                       1
Mario Kart: Super Circuit                     1
Name: Name, Length: 223, dtype: int64

In [23]:
nome = base.Name

In [24]:
base = base.drop("Name", axis=1)
base.shape

(258, 12)

### Dividindo o Dataset entre previsores e classes

In [25]:
previsores = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values

In [26]:
vendas_na = base.iloc[:, 4].values
vendas_eu = base.iloc[:, 5].values
vendas_jp = base.iloc[:, 6].values

### Aplicando o LabelEncoder para transformar atribultos categoricos em numéricos

In [32]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder = LabelEncoder()
previsores[:, 0] = labelencoder.fit_transform(previsores[:, 0])
previsores[:, 2] = labelencoder.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder.fit_transform(previsores[:, 3])
previsores[:, 8] = labelencoder.fit_transform(previsores[:, 8])


In [33]:
print(previsores)

[[10 2006.0 10 ... '8' 322.0 0]
 [10 2008.0 6 ... '8.3' 709.0 0]
 [10 2009.0 10 ... '8' 192.0 0]
 ...
 [10 2007.0 10 ... '8' 124.0 1]
 [6 2001.0 6 ... '7.9' 46.0 3]
 [6 2003.0 9 ... '8.4' 42.0 3]]


### Aplicação do OneHotEncoder, para que nunhum atributo seja mais relevante que o outro

In [34]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", 
                                                  OneHotEncoder(), [0,2,3,8])],
                                                  remainder='passthrough')

previsores = onehotencoder.fit_transform(previsores).toarray()

In [35]:
previsores.shape

(258, 61)

# Criando o modelo de Machine Learning com o Keras

In [36]:
camada_entrada = Input(shape=(61,))
camada_oculta1 = Dense(units = 32, activation = 'sigmoid')(camada_entrada)
camada_oculta2 = Dense(units = 32, activation = 'sigmoid')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [38]:
regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1, camada_saida2, camada_saida3])

regressor.compile(optimizer = 'adam',
                  loss = 'mse')

regressor.fit(previsores, [vendas_na, vendas_eu, vendas_jp],
              epochs = 5000, batch_size = 100)

previsao_na, previsao_eu, previsao_jp = regressor.predict(previsores)


Epoch 1/5000
3/3 [==============================] - 3s 12ms/step - loss: 28.3138 - dense_2_loss: 17.6268 - dense_3_loss: 9.2195 - dense_4_loss: 1.4675
Epoch 2/5000
3/3 [==============================] - 0s 9ms/step - loss: 27.5599 - dense_2_loss: 17.4897 - dense_3_loss: 8.7545 - dense_4_loss: 1.3158
Epoch 3/5000
3/3 [==============================] - 0s 7ms/step - loss: 26.4315 - dense_2_loss: 16.4807 - dense_3_loss: 8.6117 - dense_4_loss: 1.3392
Epoch 4/5000
3/3 [==============================] - 0s 12ms/step - loss: 26.8364 - dense_2_loss: 17.2217 - dense_3_loss: 8.3905 - dense_4_loss: 1.2241
Epoch 5/5000
3/3 [==============================] - 0s 10ms/step - loss: 30.5624 - dense_2_loss: 19.3421 - dense_3_loss: 9.9863 - dense_4_loss: 1.2340
Epoch 6/5000
3/3 [==============================] - 0s 8ms/step - loss: 24.4668 - dense_2_loss: 15.7133 - dense_3_loss: 7.4896 - dense_4_loss: 1.2639
Epoch 7/5000
3/3 [==============================] - 0s 10ms/step - loss: 28.9971 - dense_2_loss: 

Epoch 55/5000
3/3 [==============================] - 0s 10ms/step - loss: 23.1297 - dense_2_loss: 14.8615 - dense_3_loss: 7.1399 - dense_4_loss: 1.1283
Epoch 56/5000
3/3 [==============================] - 0s 9ms/step - loss: 22.5861 - dense_2_loss: 14.4466 - dense_3_loss: 6.9550 - dense_4_loss: 1.1845
Epoch 57/5000
3/3 [==============================] - 0s 6ms/step - loss: 22.5675 - dense_2_loss: 14.4203 - dense_3_loss: 7.0391 - dense_4_loss: 1.1081
Epoch 58/5000
3/3 [==============================] - 0s 15ms/step - loss: 23.0937 - dense_2_loss: 14.6563 - dense_3_loss: 7.2612 - dense_4_loss: 1.1762
Epoch 59/5000
3/3 [==============================] - 0s 6ms/step - loss: 14.9315 - dense_2_loss: 9.4164 - dense_3_loss: 4.5193 - dense_4_loss: 0.9958
Epoch 60/5000
3/3 [==============================] - 0s 9ms/step - loss: 17.0396 - dense_2_loss: 10.8423 - dense_3_loss: 5.1368 - dense_4_loss: 1.0606
Epoch 61/5000
3/3 [==============================] - 0s 8ms/step - loss: 20.5137 - dense_2_lo

3/3 [==============================] - 0s 8ms/step - loss: 17.3715 - dense_2_loss: 10.9522 - dense_3_loss: 5.4467 - dense_4_loss: 0.9726
Epoch 110/5000
3/3 [==============================] - 0s 12ms/step - loss: 14.6798 - dense_2_loss: 8.8839 - dense_3_loss: 4.6165 - dense_4_loss: 1.1795
Epoch 111/5000
3/3 [==============================] - 0s 9ms/step - loss: 23.1183 - dense_2_loss: 14.6189 - dense_3_loss: 7.2324 - dense_4_loss: 1.2671
Epoch 112/5000
3/3 [==============================] - ETA: 0s - loss: 13.9292 - dense_2_loss: 8.2349 - dense_3_loss: 4.5574 - dense_4_loss: 1.136 - 0s 9ms/step - loss: 15.8410 - dense_2_loss: 9.8038 - dense_3_loss: 4.9107 - dense_4_loss: 1.1266
Epoch 113/5000
3/3 [==============================] - 0s 13ms/step - loss: 16.7975 - dense_2_loss: 10.4583 - dense_3_loss: 5.2821 - dense_4_loss: 1.0572
Epoch 114/5000
3/3 [==============================] - 0s 5ms/step - loss: 18.2330 - dense_2_loss: 11.2810 - dense_3_loss: 5.7780 - dense_4_loss: 1.1740
Epoch 115

3/3 [==============================] - 0s 9ms/step - loss: 23.4436 - dense_2_loss: 15.0246 - dense_3_loss: 7.1798 - dense_4_loss: 1.2392
Epoch 162/5000
3/3 [==============================] - 0s 10ms/step - loss: 22.4937 - dense_2_loss: 14.1809 - dense_3_loss: 7.1465 - dense_4_loss: 1.1663
Epoch 163/5000
3/3 [==============================] - 0s 7ms/step - loss: 16.9787 - dense_2_loss: 10.5531 - dense_3_loss: 5.2810 - dense_4_loss: 1.1446
Epoch 164/5000
3/3 [==============================] - 0s 7ms/step - loss: 16.4615 - dense_2_loss: 10.1972 - dense_3_loss: 5.1491 - dense_4_loss: 1.1151
Epoch 165/5000
3/3 [==============================] - 0s 11ms/step - loss: 16.3402 - dense_2_loss: 10.4785 - dense_3_loss: 4.7444 - dense_4_loss: 1.1172
Epoch 166/5000
3/3 [==============================] - 0s 6ms/step - loss: 14.2869 - dense_2_loss: 8.9077 - dense_3_loss: 4.2780 - dense_4_loss: 1.1013
Epoch 167/5000
3/3 [==============================] - 0s 7ms/step - loss: 21.8579 - dense_2_loss: 14.1

3/3 [==============================] - 0s 9ms/step - loss: 21.7187 - dense_2_loss: 13.7472 - dense_3_loss: 6.8442 - dense_4_loss: 1.1273
Epoch 215/5000
3/3 [==============================] - 0s 8ms/step - loss: 15.0942 - dense_2_loss: 9.2597 - dense_3_loss: 4.6552 - dense_4_loss: 1.1793
Epoch 216/5000
3/3 [==============================] - 0s 9ms/step - loss: 17.2854 - dense_2_loss: 10.9436 - dense_3_loss: 5.2999 - dense_4_loss: 1.0419
Epoch 217/5000
3/3 [==============================] - 0s 8ms/step - loss: 22.2857 - dense_2_loss: 14.2093 - dense_3_loss: 6.9175 - dense_4_loss: 1.1588
Epoch 218/5000
3/3 [==============================] - 0s 7ms/step - loss: 14.1967 - dense_2_loss: 8.8545 - dense_3_loss: 4.1256 - dense_4_loss: 1.2166
Epoch 219/5000
3/3 [==============================] - 0s 9ms/step - loss: 15.4273 - dense_2_loss: 9.3947 - dense_3_loss: 4.7163 - dense_4_loss: 1.3163
Epoch 220/5000
3/3 [==============================] - 0s 12ms/step - loss: 22.4645 - dense_2_loss: 14.2189

...

Epoch 4771/5000
3/3 [==============================] - 0s 9ms/step - loss: 6.5861 - dense_2_loss: 4.0087 - dense_3_loss: 1.7747 - dense_4_loss: 0.8027
Epoch 4772/5000
3/3 [==============================] - 0s 12ms/step - loss: 6.7021 - dense_2_loss: 3.9380 - dense_3_loss: 1.8780 - dense_4_loss: 0.8861
Epoch 4773/5000
3/3 [==============================] - 0s 7ms/step - loss: 6.8203 - dense_2_loss: 3.9065 - dense_3_loss: 1.9939 - dense_4_loss: 0.9199
Epoch 4774/5000
3/3 [==============================] - 0s 11ms/step - loss: 5.9993 - dense_2_loss: 3.2267 - dense_3_loss: 1.9386 - dense_4_loss: 0.8340
Epoch 4775/5000
3/3 [==============================] - 0s 8ms/step - loss: 5.9335 - dense_2_loss: 3.4538 - dense_3_loss: 1.6446 - dense_4_loss: 0.8351
Epoch 4776/5000
3/3 [==============================] - ETA: 0s - loss: 6.2858 - dense_2_loss: 3.4252 - dense_3_loss: 1.8890 - dense_4_loss: 0.97 - 0s 8ms/step - loss: 6.4364 - dense_2_loss: 3.6503 - dense_3_loss: 1.8495 - dense_4_loss: 0.9366


3/3 [==============================] - 0s 10ms/step - loss: 6.6192 - dense_2_loss: 3.8444 - dense_3_loss: 1.8530 - dense_4_loss: 0.9218
Epoch 4824/5000
3/3 [==============================] - 0s 8ms/step - loss: 5.1431 - dense_2_loss: 2.8681 - dense_3_loss: 1.4667 - dense_4_loss: 0.8083
Epoch 4825/5000
3/3 [==============================] - 0s 8ms/step - loss: 5.9951 - dense_2_loss: 3.5444 - dense_3_loss: 1.6548 - dense_4_loss: 0.7958
Epoch 4826/5000
3/3 [==============================] - 0s 5ms/step - loss: 5.6622 - dense_2_loss: 3.3179 - dense_3_loss: 1.5961 - dense_4_loss: 0.7482
Epoch 4827/5000
3/3 [==============================] - 0s 9ms/step - loss: 6.3524 - dense_2_loss: 3.6453 - dense_3_loss: 1.7627 - dense_4_loss: 0.9444
Epoch 4828/5000
3/3 [==============================] - 0s 7ms/step - loss: 6.0960 - dense_2_loss: 3.4097 - dense_3_loss: 1.8949 - dense_4_loss: 0.7914
Epoch 4829/5000
3/3 [==============================] - 0s 10ms/step - loss: 6.1216 - dense_2_loss: 3.3973 - d

3/3 [==============================] - 0s 6ms/step - loss: 5.7072 - dense_2_loss: 3.3687 - dense_3_loss: 1.5541 - dense_4_loss: 0.7844
Epoch 4877/5000
3/3 [==============================] - 0s 9ms/step - loss: 5.1341 - dense_2_loss: 2.6577 - dense_3_loss: 1.6741 - dense_4_loss: 0.8022
Epoch 4878/5000
3/3 [==============================] - 0s 9ms/step - loss: 5.5399 - dense_2_loss: 3.0158 - dense_3_loss: 1.6837 - dense_4_loss: 0.8405
Epoch 4879/5000
3/3 [==============================] - 0s 7ms/step - loss: 5.4052 - dense_2_loss: 2.9647 - dense_3_loss: 1.6704 - dense_4_loss: 0.7700
Epoch 4880/5000
3/3 [==============================] - 0s 5ms/step - loss: 5.5214 - dense_2_loss: 2.9165 - dense_3_loss: 1.6811 - dense_4_loss: 0.9237
Epoch 4881/5000
3/3 [==============================] - 0s 20ms/step - loss: 6.1923 - dense_2_loss: 3.5306 - dense_3_loss: 1.7974 - dense_4_loss: 0.8643
Epoch 4882/5000
3/3 [==============================] - 0s 10ms/step - loss: 5.9197 - dense_2_loss: 3.2680 - d

3/3 [==============================] - 0s 11ms/step - loss: 6.3033 - dense_2_loss: 3.7055 - dense_3_loss: 1.7378 - dense_4_loss: 0.8600
Epoch 4930/5000
3/3 [==============================] - 0s 8ms/step - loss: 5.9340 - dense_2_loss: 3.1858 - dense_3_loss: 1.9026 - dense_4_loss: 0.8457
Epoch 4931/5000
3/3 [==============================] - 0s 7ms/step - loss: 5.3801 - dense_2_loss: 2.9062 - dense_3_loss: 1.6012 - dense_4_loss: 0.8726
Epoch 4932/5000
3/3 [==============================] - 0s 5ms/step - loss: 5.3349 - dense_2_loss: 2.9183 - dense_3_loss: 1.5681 - dense_4_loss: 0.8485
Epoch 4933/5000
3/3 [==============================] - 0s 10ms/step - loss: 6.0581 - dense_2_loss: 3.3459 - dense_3_loss: 1.9170 - dense_4_loss: 0.7953
Epoch 4934/5000
3/3 [==============================] - 0s 12ms/step - loss: 5.7016 - dense_2_loss: 3.1421 - dense_3_loss: 1.6796 - dense_4_loss: 0.8799
Epoch 4935/5000
3/3 [==============================] - 0s 6ms/step - loss: 6.0671 - dense_2_loss: 3.5468 - 

3/3 [==============================] - 0s 6ms/step - loss: 5.7535 - dense_2_loss: 3.1058 - dense_3_loss: 1.7417 - dense_4_loss: 0.9061
Epoch 4982/5000
3/3 [==============================] - 0s 11ms/step - loss: 5.2457 - dense_2_loss: 2.8673 - dense_3_loss: 1.4706 - dense_4_loss: 0.9078
Epoch 4983/5000
3/3 [==============================] - 0s 8ms/step - loss: 5.1394 - dense_2_loss: 2.6779 - dense_3_loss: 1.5916 - dense_4_loss: 0.8699
Epoch 4984/5000
3/3 [==============================] - 0s 9ms/step - loss: 6.0628 - dense_2_loss: 3.2274 - dense_3_loss: 1.8732 - dense_4_loss: 0.9622
Epoch 4985/5000
3/3 [==============================] - 0s 5ms/step - loss: 5.5810 - dense_2_loss: 3.0239 - dense_3_loss: 1.6995 - dense_4_loss: 0.8577
Epoch 4986/5000
3/3 [==============================] - 0s 6ms/step - loss: 5.2442 - dense_2_loss: 2.8598 - dense_3_loss: 1.5891 - dense_4_loss: 0.7953
Epoch 4987/5000
3/3 [==============================] - 0s 7ms/step - loss: 5.9680 - dense_2_loss: 3.5663 - de